In [1]:
words = ["abogado","aceite","áfrica","agente","agua","águila","aguja","agujero","aire","alemania","algodón","alianza","alpes","ambulancia","américa","ángel","anillo","antártida","antorcha","araña","archivo","arco","argentina","artículo","as","atlántida","azteca","baile","bala","ballena","banco","banda","baño","barco","barra","batería","berlín","bermudas","bicho","blanco","bloque","boca","bola","bolsa","bomba","bosque","bota","botella","botón","brazo","bruja","caballero","caballo","cabeza","cabina","cabo","cactus","cadena","caja","cama","cámara","cambio","campana","campo","canal","canguro","canto","caña","capa","capital","caqui","cara","caravana","carga","carrera","carro","carta","casco","casino","caza","cementerio","centauro","centro","cervantes","checo","chocolate","choque","chuleta","científico","cinta","cinturón","círculo","clase","coche","cocinero","coco","código","cola","cólera","columna","cometa","compás","concierto","conejo","contrabandista","copa","corazón","corneta","corona","corredor","corriente","corte","cresta","cromo","cruz","cuadro","cuarto","cubierta","cubo","cuchillo","cuello","cuerda","cuerno","cura","dama","delta","destino","día","diamante","diana","diario","diente","dinosaurio","disco","don","dragón","duende","egipto","embajada","emperador","enano","enfermedad","enfermera","enlace","escorpión","espacio","espía","estación","estadio","estado","estrella","estudio","etiqueta","europa","extraterrestre","falda","fantasma","faro","ficha","fiesta","figura","flauta","flecha","foso","francia","frente","fuego","fuente","fuerza","furgoneta","gancho","gato","genio","gigante","golfo","golondrina","golpe","goma","góndola","gota","grado","granada","grano","grecia","grifo","guante","guardia","guerra","gusano","helado","helicóptero","hielo","hierba","hoja","hollywood","horca","hospital","hotel","iglesia","imán","india","índice","inglaterra","italia","jarra","judía","juicio","kiwi","ladrón","lago ness","láser","látigo","lengua","león","libra","lima","limusina","línea","lista","llama","llave","lomo","londres","luna","luz","maestro","magia","malta","mancha","mando","manga","mango","mano","manzana","mañana","marca","marcha","marfil","masa","máscara","mazo","médico","mercurio","mesa","metro","méxico","micro","microscopi","mielo","millonario","mina","misil","modelo","módulo","monitor","mono","mortero","moscú","motor","muelle","muerte","muñeca","muro","naranja","nave","nieve","nilo","ninja","noche","nota","nudo","nueva york","obra","ojo","ola","olimpo","ópera","orden","órgano","ornitorrinco","oro","oso","pala","palma","pantalla","papel","paracaídas","pase","paso","pasta","pastel","pavo","pekín","película","pelotón","pendiente","perro","pez","pico","pie","pieza","pila","piloto","pincho","pingüino","pinta","piña","pirámide","pirata","pista","pistola","placa","plano","planta","plátano","playa","plomo","pluma","policía","polo","portada","portero","potro","prensa","prima","princesa","puente","puerto","pulpo","pulso","punta","punto","radio","rascacielos","ratón","rayo","red","regla","reina","reserva","revolución","rey","robot","rojo","roma","ronda","rosa","ruleta","sable","sáhara","salsa","satélite","saturno","señal","serie","serpiente","sierra","silla","sirena","sobre","soldado","submarinista","suerte","superhéroe","tabla","tableta","taco","tacto","talón","tanque","tapa","tarde","teatro","teclado","telescopio","testigo","tiempo","tienda","tierra","tokio","topo","torre","trama","tronco","tubería","tubo","unicornio","vacío","vado","vampiro","vela","veneno","venus","vestido","vida","vidrio","viento","yema","zanahoria","zapato"]

In [2]:
import fasttext
import fasttext.util
import gensim
from gensim.models import FastText, Word2Vec
from gensim.models.fasttext import load_facebook_model
from nltk.corpus import wordnet as wn
import nltk
import itertools
import pandas as pd

In [ ]:
nltk.download("omw-1.4")
nltk.download("wordnet")

In [12]:
def get_cosine_similarity(model, word1, word2):
    return model.wv.similarity(word1, word2)

def get_synonyms(word1, lang = 'spa'):
    all_synonyms = list(map(lambda x: x.lemma_names(lang), wn.synsets(word1, lang=lang)))
    return list(set(itertools.chain(*all_synonyms)))

def get_antonyms(word1, lang = 'spa'):
    all_lemas = list(map(lambda x: x.lemmas(), wn.synsets(word1, lang=lang)))
    all_antonyms_lemas = list(map(lambda x: x.antonyms(), list(itertools.chain(*all_lemas))))
    all_antonyms = map(lambda x: x.synset().lemma_names(lang=lang), list(itertools.chain(*all_antonyms_lemas)))
    return list(set(itertools.chain(*all_antonyms)))

def get_synonyms_with_similarity(word1, model, lang='spa'):
    return list(map(lambda word2: (word2, get_cosine_similarity(model, word1, word2)), get_synonyms(word1, lang=lang)))

def get_max_synonym_similarity_between_words(word1, word2, model, lang='spa'):
    synonyms_word1 = get_synonyms(word1, lang = lang)
    synonyms_word2 = get_synonyms(word2, lang = lang)
    synonyms = list(set(synonyms_word1).intersection(synonyms_word2))
    if len(synonyms) !=0:
        max_value = max(synonyms)
    else:
        max_value = 0
    max_value = max_value if max_value else 0
    return max_value

def get_number_of_same_synonyms(word1, word2, lang = 'spa'):
    synonyms_word1 = get_synonyms(word1, lang = lang)
    synonyms_word2 = get_synonyms(word2, lang = lang)
    return len(set(synonyms_word1).intersection(synonyms_word2))

def get_number_of_same_antonyms(word1, word2, lang = 'spa'):
    antonyms_word1 = get_antonyms(word1, lang = lang)
    antonyms_word2 = get_antonyms(word2, lang = lang)
    return len(set(antonyms_word1).intersection(antonyms_word2))

def get_number_synonyms_vs_antonyms(word1, word2, lang = 'spa'):
    antonyms_word1 = get_antonyms(word1, lang = lang)
    synonyms_word1 = get_synonyms(word1, lang = lang)
    antonyms_word2 = get_antonyms(word2, lang = lang)
    synonyms_word2 = get_synonyms(word2, lang = lang)
    return len(set(synonyms_word1).intersection(antonyms_word2)) + len(set(antonyms_word1).intersection(synonyms_word2))

def get_antonyms_with_similarity(word1, model, lang='spa'):
    return list(map(lambda word2: (word2, get_cosine_similarity(model, word1, word2)), get_antonyms(word1, lang=lang)))

def get_max_antonym_similarity_between_words(word1, word2, model, lang='spa'):
    antonyms_word1 = get_antonyms(word1, lang = lang)
    antonyms_word2 = get_antonyms(word2, lang = lang)
    antonyms = list(set(antonyms_word1).intersection(antonyms_word2))
    if len(antonyms) != 0:
        max_value = max(antonyms)
    else:
        max_value = 0
    max_value = max_value if max_value else 0
    return max_value

def get_hypernyms_of_synsets(synsets):
    hypernyms = list(map(lambda x: x.hypernyms(), synsets))
    return list(set(itertools.chain(*hypernyms)))

def get_path_similarity(synset1, synset2):
    similarity = wn.path_similarity(synset1, synset2)
    similarity = similarity if similarity else 0
    return similarity

def get_first_common_hypernym(word1, word2, lang = 'spa'):
    temporal_hypernyms1 = get_hypernyms_of_synsets(wn.synsets(word1, lang=lang))
    temporal_hypernyms2 = get_hypernyms_of_synsets(wn.synsets(word2, lang=lang))
    hypernyms1 = list(set(temporal_hypernyms1))
    hypernyms2 = list(set(temporal_hypernyms2))
    while len(set(hypernyms1).intersection(hypernyms2))==0 and (len(temporal_hypernyms1)!=0 or len(temporal_hypernyms2)!=0):
        hypernyms1 = list(set(hypernyms1 + temporal_hypernyms1))
        hypernyms2 = list(set(hypernyms2 + temporal_hypernyms2))
        temporal_hypernyms1 = get_hypernyms_of_synsets(temporal_hypernyms1)
        temporal_hypernyms2 = get_hypernyms_of_synsets(temporal_hypernyms2)
    if len(set(hypernyms1).intersection(hypernyms2))!=0:
        hypernyms = list(set(hypernyms1).intersection(hypernyms2))
        possible_hypernyms = list(map(lambda x: max(list(map(lambda y: get_path_similarity(x, y), wn.synsets(word1, lang=lang)))) + max(list(map(lambda y: get_path_similarity(x, y), wn.synsets(word2, lang=lang)))),hypernyms))
        max_value = max(possible_hypernyms)
        hypernym = hypernyms[possible_hypernyms.index(max_value)]
        similarity_to_word1 = max(list(map(lambda x: get_path_similarity(hypernyms[possible_hypernyms.index(max_value)],x),wn.synsets(word1, lang=lang))))
        similarity_to_word2 = max(list(map(lambda x: get_path_similarity(hypernyms[possible_hypernyms.index(max_value)],x),wn.synsets(word2, lang=lang))))
    else:
        hypernym = None
        similarity_to_word1 = 0
        similarity_to_word2 = 0
    return hypernym, similarity_to_word1, similarity_to_word2
    
    
def apply_all_metrics_to_words_df(df, model, word2, lang = 'spa'):
    df['similarity'] = df.apply(lambda x: get_cosine_similarity(model, x['words'], word2), axis=1)
    df['number_same_synonyms'] = df.apply(lambda x: get_number_of_same_synonyms(x['words'], word2, lang= lang), axis=1)
    df['number_same_antonyms'] = df.apply(lambda x: get_number_of_same_antonyms(x['words'], word2, lang= lang), axis=1)
    df['synonyms_vs_antonyms'] = df.apply(lambda x: get_number_synonyms_vs_antonyms(x['words'], word2, lang= lang), axis=1)
    df['max_similarity_synonym'] = df.apply(lambda x: get_max_synonym_similarity_between_words(x['words'], word2, model, lang=lang), axis=1)
    df['max_similarity_antonym'] = df.apply(lambda x: get_max_antonym_similarity_between_words(x['words'], word2, model, lang=lang), axis=1)
    df['distance_common_hypernym'] =  df.apply(lambda x: get_first_common_hypernym(x['words'], word2, lang= lang)[1], axis=1)
    df['distance_common_hypernym_from_word'] =  df.apply(lambda x: get_first_common_hypernym(x['words'], word2, lang= lang)[2], axis=1)
    return df

def get_definition(word, lang = 'spa'):
    return list(map(lambda x: x.definition(),wn.synsets(word, lang= lang)))

In [4]:
fasttext.util.download_model('es', if_exists='ignore')  # English

'cc.es.300.bin'

In [5]:
%%capture
FAST_TEXT_MODEL = "cc.es.300.bin" # Model name in fasttext


ft = fasttext.load_model(FAST_TEXT_MODEL)

In [ ]:
#Grado similaridad
#Numero de sinonimos en común
#Número de antónimos en común
#Relación con los sinonimos
#Relación con los antónimos
#Familia semántica (Hiperonimos)
#Número de palabras entre la pista y la evaluada(s)


# Implementar distancia de la definiciones
# Hacer clustering (jerarquico y k-means)
# Implementar bots

In [10]:
model = load_facebook_model(FAST_TEXT_MODEL)
#model.wv.most_similar(positive=['red', 'green'])

In [6]:
data = pd.DataFrame(words, columns = ["words"])

In [7]:
sample_data = data.sample(25)

In [8]:
sample_data

,words
147,estación
364,tabla
375,testigo
73,carga
132,dinosaurio
315,playa
309,pista
296,perro
183,grecia
257,moscú


In [11]:
apply_all_metrics_to_words_df(sample_data, model, 'animal')

,words,similarity,number_same_synonyms,number_same_antonyms,synonyms_vs_antonyms,max_similarity_synonym,max_similarity_antonym,distance_common_hypernym,distance_common_hypernym_from_word
147,estación,0.126598,0,0,0,0,0,0.250000,0.250000
364,tabla,0.040919,0,0,0,0,0,0.250000,0.250000
375,testigo,0.201387,0,0,0,0,0,0.333333,0.500000
73,carga,0.178553,0,0,0,0,0,0.250000,0.250000
132,dinosaurio,0.492935,0,0,0,0,0,0.125000,0.500000
315,playa,0.074658,0,0,0,0,0,0.333333,0.200000
309,pista,0.138136,0,0,0,0,0,0.250000,0.250000
296,perro,0.647896,0,0,0,0,0,0.250000,0.500000
183,grecia,0.091543,0,0,0,0,0,0.000000,0.000000
257,moscú,0.063005,0,0,0,0,0,0.000000,0.000000


In [25]:
get_definition('perro')

['a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds',
 'a person who is deemed to be despicable or contemptible']

In [ ]:
ft.get_nearest_neighbors('life')

In [ ]:
model.wv.similarity('casa', 'hogar')

In [ ]:
a = list(map(lambda x: x.lemma_names('spa'), wn.synsets('casa', lang='spa')))
set(itertools.chain(*a))

In [ ]:
get_antonyms('bien', lang='spa')

In [ ]:
get_number_synonyms_vs_antonyms('bien', 'mal', lang='spa')

In [ ]:
first = wn.synset('apple.n.01')
second = wn.synset('banana.n.01')

In [ ]:
fist = first.hypernyms()

In [ ]:
fist = list(map(lambda x: x.hypernyms(),list(set(itertools(*fist)))))

In [ ]:
get_first_common_hypernym('manzana', 'pera', lang ='spa')

In [15]:
get_hypernyms_of_synsets(wn.synsets('piña', lang='spa'))

[]

In [23]:
wn.synsets('rhino')

[Synset('rhinoceros.n.01')]

In [29]:
get_cosine_similarity(model, 'manzana','plátano')

0.5180584

In [ ]:
wn.synsets('pera', lang ='spa')

In [ ]:
wn.path_similarity(wn.synset('physical_entity.n.01'), wn.synset('well.r.13'))